<a href="https://colab.research.google.com/github/Shubham-M-Rathod/twitter-analysis-disaster-dataset/blob/main/nlp_twitter_dataset_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [75]:
data = pd.read_csv('train.csv')

In [76]:
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [77]:
data[data['text'].duplicated()==True]

,id,keyword,location,text,target
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0
115,165,aftershock,US,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...,0
119,172,aftershock,Switzerland,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/TH...,0
164,238,airplane%20accident,NaN,Experts in France begin examining airplane deb...,1
624,898,bioterrorism,NaN,To fight bioterrorism sir.,0
...,...,...,...,...,...
7600,10855,NaN,NaN,Evacuation order lifted for town of Roosevelt:...,1
7607,10867,NaN,NaN,#stormchase Violent Record Breaking EF-5 El Re...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1


In [78]:
data = data.drop_duplicates(subset=['text'])

In [8]:
data['keyword'] = data['keyword'].fillna('')

<ipython-input-8-3edc8f8650b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keyword'] = data['keyword'].fillna('')


In [ ]:
data['keyword'] = data['keyword'].str.replace('%20', ' ')

In [ ]:
import nltk
nltk.download('stopwords')

In [136]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import string
sno = SnowballStemmer(language='english')

def preprocess(s):
  s = s.lower().translate(str.maketrans('', '', string.punctuation))
  processed_sentence =[sno.stem(word) for word in s.split() if word not in stopwords.words('english') \
                                 and not word.startswith(('www', 'http', 'https','@'))]
  processed_sentence = ([word[1:] if word.startswith('#') else word for word in processed_sentence])
  processed_sentence =  ' '.join([word for word in processed_sentence if word.isalpha()])
  return processed_sentence

In [139]:
print(data['text'][87],'\n')
print(preprocess(data['text'][87]))

RT nAAYf: First accident in years. Turning onto Chandanee Magu from near MMA. Taxi rammed into me while I was halfway turned. Everyone confÛ_ 

rt naayf first accid year turn onto chandane magu near mma taxi ram halfway turn everyon


In [ ]:
data['final_text'] = data['text'].apply(preprocess)

In [ ]:
data.final_text = data.final_text.fillna(' ')
data.final_text.isna().sum()

In [32]:
list_of_sent=[]
for sent in data['final_text'].values:
  list_of_sent.append([word for word in sent.split()])

In [40]:
import gensim
model = gensim.models.Word2Vec(list_of_sent, vector_size=50, min_count=2)

In [41]:
len(model.wv)

5109

In [56]:
model.wv.most_similar('gun')

[('damag', 0.9984451532363892),
 ('area', 0.9983832836151123),
 ('desol', 0.998330295085907),
 ('day', 0.9983295798301697),
 ('im', 0.9982794523239136),
 ('via', 0.9982644319534302),
 ('polic', 0.9982597827911377),
 ('rt', 0.9982547163963318),
 ('still', 0.9982514381408691),
 ('amp', 0.9982168078422546)]

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tfidf = tf_idf_vect.fit_transform(data['final_text'].values)

In [62]:
tfidf_feat = tf_idf_vect.get_feature_names_out()
len(tfidf_feat)

57625

In [80]:
tfidf_wv_sent = []
row=0
for sent in list_of_sent:
  sent_vec = np.zeros(50)
  sum_tfidf = 0
  for word in sent:
    if word in model.wv and word in tfidf_feat:
      vec = model.wv[word]
      tfidf = final_tfidf[row, tfidf_feat.tolist().index(word)]
      sent_vec += vec*tfidf
      sum_tfidf += tfidf
  if sum_tfidf !=0:
    sent_vec /= sum_tfidf
  tfidf_wv_sent.append(sent_vec)
  row+=1